In [1]:
!pip install -qqq transformers

In [2]:
#from huggingface_hub import login
#login("set_your_token")


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchai

In [4]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.document_loaders import (
    PyPDFLoader, 
    TextLoader, 
    CSVLoader,
    Docx2txtLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import re
import json
from typing import List, Dict, Any


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-05-22 17:41:43.406117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747935703.587708      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747935703.641166      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
!pip install faiss-gpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [7]:
!pip install torch transformers langchain langchain_community faiss-cpu sentence-transformers docx2txt pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [8]:
# Initialize the embedding model with better configuration
embedding_model_name = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}  # This helps with retrieval quality
)

/tmp/ipykernel_35/1522725409.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def clean_pdf_text(text):
    """Clean PDF text to improve quality"""
    # Remove excessive whitespace and newlines
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r' +', ' ', text)
    # Remove page numbers and headers/footers (simple heuristic)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        # Skip very short lines that are likely page numbers or artifacts
        if len(line) > 10 and not re.match(r'^\d+$', line):
            cleaned_lines.append(line)
    return '\n'.join(cleaned_lines)

In [10]:
def load_documents(file_paths):
    """Load documents from various file formats with better error handling"""
    documents = []
    for file_path in file_paths:
        file_extension = os.path.splitext(file_path)[1].lower()
        try:
            if file_extension == '.pdf':
                loader = PyPDFLoader(file_path)
                docs = loader.load()
                # Clean up PDF text which often has formatting issues
                for doc in docs:
                    doc.page_content = clean_pdf_text(doc.page_content)
                documents.extend(docs)
            elif file_extension == '.txt':
                loader = TextLoader(file_path, encoding='utf-8')
                documents.extend(loader.load())
            elif file_extension in ['.csv', '.tsv']:
                loader = CSVLoader(file_path)
                documents.extend(loader.load())
            elif file_extension in ['.docx', '.doc']:
                loader = Docx2txtLoader(file_path)
                documents.extend(loader.load())
            else:
                print(f"Unsupported file format: {file_extension}")
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
    return documents

In [11]:
def split_documents(documents, chunk_size=800, chunk_overlap=150):
    """Split documents into chunks with better parameters for accuracy"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]  # Better separators for natural breaks
    )
    return text_splitter.split_documents(documents)

In [12]:
def create_vector_store(documents):
    """Create a vector store from the documents"""
    return FAISS.from_documents(documents, embeddings)

def retrieve_context(vector_store, query, k=5):  # Increased k for better context
    """Retrieve relevant documents based on the query with better search"""
    # Use both similarity search and MMR for diversity
    docs_similarity = vector_store.similarity_search(query, k=k)
    
    # Get diverse results using Maximum Marginal Relevance
    try:
        docs_mmr = vector_store.max_marginal_relevance_search(
            query, k=k, fetch_k=k*2, lambda_mult=0.5
        )
        # Combine and deduplicate
        all_docs = docs_similarity + docs_mmr
        seen = set()
        unique_docs = []
        for doc in all_docs:
            content_hash = hash(doc.page_content)
            if content_hash not in seen:
                seen.add(content_hash)
                unique_docs.append(doc)
        return unique_docs[:k]
    except:
        return docs_similarity

In [13]:
def format_context_for_llm(docs):
    """Format retrieved documents as context for the LLM with better structure"""
    if not docs:
        return "No relevant documents found."
    
    context = "Here are the relevant documents that may help answer your question:\n\n"
    for i, doc in enumerate(docs):
        content = doc.page_content.strip()
        source = doc.metadata.get('source', 'Unknown source')
        page = doc.metadata.get('page', '')
        
        context += f"=== Document {i+1} ===\n"
        context += f"Source: {os.path.basename(source)}"
        if page:
            context += f" (Page {page})"
        context += f"\nContent: {content}\n\n"
    
    return context

In [14]:
def clean_response(response):
    """Clean up response by removing any special tokens or formatting"""
    response = re.sub(r'<\|im_end\|>.*', '', response)
    response = re.sub(r'<\|.*?\|>', '', response)
    response = response.strip()
    # Remove any repeated patterns or artifacts
    lines = response.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line and not line.startswith('=') and len(line) > 5:
            cleaned_lines.append(line)
    return '\n'.join(cleaned_lines)

In [15]:
def generate_llm_response(prompt, max_new_tokens=1000, temperature=0.3):
    """Generate a response with better parameters for accuracy"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4000)
    
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    input_length = inputs["input_ids"].shape[1]
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=40,
            top_p=0.9,
            temperature=temperature,  # Lower temperature for more focused responses
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return clean_response(generated_text)

In [16]:
def parse_action(text):
    """Parse the action and action input from the model's reasoning step"""
    action_match = re.search(r'Action:\s*(\w+)', text, re.IGNORECASE)
    action_input_match = re.search(r'Action Input:\s*(.*?)(?=\n(?:Thought|Action|Observation)|$)', text, re.DOTALL | re.IGNORECASE)
    
    if action_match:
        action = action_match.group(1).strip()
        action_input = action_input_match.group(1).strip() if action_input_match else ""
        return action, action_input
    return None, None

In [17]:
def execute_search(query, vector_store, k=5):
    """Tool/Action: Search the vector store for relevant information"""
    docs = retrieve_context(vector_store, query, k=k)
    context = format_context_for_llm(docs)
    return context

In [18]:
class ImprovedReActAgent:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.tools = {
            "Search": execute_search
        }
        self.max_iterations = 4
        self.conversation_history = []

    def add_to_history(self, role, content):
        """Add a message to the conversation history"""
        self.conversation_history.append({"role": role, "content": content})
    
    def format_history(self):
        """Format the conversation history for the prompt"""
        formatted = ""
        for msg in self.conversation_history:
            formatted += f"<|im_start|>{msg['role']}\n{msg['content']}<|im_end|>\n\n"
        return formatted
    
    def run(self, query):
        """Run the ReAct agent with improved prompting"""
        self.conversation_history = []
        
        system_prompt = """You are a helpful AI assistant that answers questions by searching through documents step by step.

Available actions:
- Search: search for relevant information in the document database
- Finish: provide a final answer based on the information found

Instructions:
1. Always search for relevant information first before answering
2. Use the retrieved information to provide accurate, specific answers
3. Cite specific details from the documents when possible
4. If you can't find relevant information, say so clearly

Format your response as:
Thought: [your reasoning about what to do next]
Action: [Search or Finish]
Action Input: [search query or final answer]

When you have found sufficient information, use:
Thought: I have found relevant information to answer the question.
Action: Finish
Action Input: [your complete answer based on the retrieved documents]"""

        self.add_to_history("system", system_prompt)
        self.add_to_history("user", f"Question: {query}")
        
        iteration = 0
        final_answer = ""
        
        while iteration < self.max_iterations:
            iteration += 1
            
            prompt = self.format_history()
            prompt += "<|im_start|>assistant\n"
            
            reasoning = generate_llm_response(prompt, max_new_tokens=800)
            action, action_input = parse_action(reasoning)
            
            self.add_to_history("assistant", reasoning)
            
            if action and action.lower() == "finish":
                final_answer = action_input
                break
            
            if action and action.lower() == "search" and action_input:
                search_result = self.tools["Search"](action_input, self.vector_store)
                observation = f"Search Results:\n{search_result}"
                self.add_to_history("system", observation)
            else:
                observation = "Please specify a valid action (Search or Finish) with appropriate input."
                self.add_to_history("system", observation)
        
        if not final_answer:
            # Force a final answer if max iterations reached
            prompt = self.format_history()
            prompt += "<|im_start|>assistant\nThought: I need to provide a final answer based on the information I've gathered.\nAction: Finish\nAction Input: "
            
            final_answer = generate_llm_response(prompt, max_new_tokens=500)
        
        return {
            "final_answer": final_answer,
            "conversation": self.conversation_history,
            "iterations": iteration
        }


In [19]:
def generate_rag_with_react(query, vector_store):
    """Generate a response using improved RAG with multi-step reasoning"""
    agent = ImprovedReActAgent(vector_store)
    result = agent.run(query)
    return result

In [20]:
!pip install fastapi uvicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00


In [21]:
!pip install python-multipart


In [22]:
# Set up ngrok
#from pyngrok import ngrok
#ngrok.set_auth_token("set_your_token")

In [23]:
import json
from typing import Dict, Any

def print_conversation_data(data: Dict[str, Any]) -> None:
    """
    Prints conversation data in a properly formatted, readable way.
    
    Args:
        data: Dictionary containing 'final_answer', 'conversation', and 'iterations'
    """
    print("=" * 80)
    print("CONVERSATION ANALYSIS REPORT")
    print("=" * 80)
    
    # Print final answer
    if 'final_answer' in data:
        print(f"\n📋 FINAL ANSWER:")
        print("-" * 40)
        print(f"{data['final_answer']}")
    
    # Print iterations count
    if 'iterations' in data:
        print(f"\n🔄 TOTAL ITERATIONS: {data['iterations']}")
    
    # Print conversation
    if 'conversation' in data and data['conversation']:
        print(f"\n💬 CONVERSATION FLOW:")
        print("-" * 40)
        
        for i, message in enumerate(data['conversation'], 1):
            role = message.get('role', 'unknown')
            content = message.get('content', '')
            
            # Format role display
            role_display = {
                'system': '🤖 SYSTEM',
                'user': '👤 USER',
                'assistant': '🤖 ASSISTANT'
            }.get(role, f'❓ {role.upper()}')
            
            print(f"\n[{i}] {role_display}:")
            print("-" * 30)
            
            # Handle different content types
            if role == 'system' and len(content) > 200:
                # Truncate long system messages
                print(f"{content[:200]}...")
                print("(System message truncated for readability)")
            elif 'Search Result:' in content:
                # Format search results nicely
                print("🔍 SEARCH RESULTS:")
                lines = content.split('\n')
                for line in lines[1:]:  # Skip "Search Result:" line
                    if line.strip():
                        if line.startswith('Document'):
                            print(f"\n📄 {line}")
                        else:
                            print(f"   {line}")
            else:
                # Regular content
                print(content)
    
    print("\n" + "=" * 80)

def load_and_print_from_string(data_input) -> None:
    """
    Loads conversation data from various input types and prints it formatted.
    
    Args:
        data_input: String, dictionary, or other object containing conversation data
    """
    try:
        # Handle different input types
        if isinstance(data_input, dict):
            # Already a dictionary
            data = data_input
        elif isinstance(data_input, str):
            # Try to parse as JSON first, then eval as fallback
            try:
                data = json.loads(data_input)
            except json.JSONDecodeError:
                try:
                    data = eval(data_input)
                except:
                    raise ValueError("Could not parse string as dictionary or JSON")
        else:
            # Try to convert to dict if it has dict-like attributes
            if hasattr(data_input, '__dict__'):
                data = data_input.__dict__
            else:
                raise ValueError(f"Unsupported data type: {type(data_input)}")
        
        print_conversation_data(data)
        
    except Exception as e:
        print(f"Error parsing data: {e}")
        print("Raw data type:", type(data_input))
        print("Raw data:")
        print(data_input)

# Example usage with your data
if __name__ == "__main__":
    # Your data string
    data_str = """{'final_answer': 'The technique used in the paper is Retrieval-Augmented Generation (RAG) with a combination of PubMed, StatPearls, and Textbooks corpora.', 'conversation': [{'role': 'system', 'content': "\\n        You are an assistant that solves problems using a step-by-step approach.\\nYou can take the following actions:\\n- Search: search for relevant information in the document database\\n- Finish: provide a final answer based on what you've learned\\n\\nFor each step, you should:\\n1. Think about what you know and what you need to find out\\n2. Decide on an action to take\\n3. Observe the results\\n4. Continue until you can provide a complete answer\\n\\nFormat your reasoning as:\\nThought: <your reasoning>\\nAction: <Search or Finish>\\nAction Input: <search query or final answer>\\n\\nWhen you have enough information to provide a final answer, use:\\nThought: I now have enough information to answer the question.\\nAction: Finish\\nAction Input: <your complete answer>"}, {'role': 'user', 'content': 'what is the techique that used in this paper?'}, {'role': 'assistant', 'content': 'Thought: I need to find out the title and authors of the paper to determine the technique used.\\nAction: Search\\nAction Input: "paper title and authors"'}, {'role': 'system', 'content': 'Search Result:\\nDocument 1 (Source: 76c4726f8736dba2.pdf):\\ntaining over 36 million biomedical articles. Many\\nrelevant studies solely use PubMed as the retrieval\\ncorpus (Frisoni et al., 2022; Naik et al., 2022). For\\nMEDRAG, we use a PubMed subset of 23.9 million\\narticles with valid titles and abstracts.\\nStatPearls. StatPearls12 is a point-of-the-care\\nclinical decision support tool similar to UpTo-\\nDate13. We use the 9,330 publicly available Stat-\\nPearl articles through NCBI Bookshelf 14 to con-\\nstruct the StatPearls corpus. We chunked StatPearls\\naccording to the hierarchical structure, treating\\neach paragraph in an article as a snippet and splic-\\ning all the relevant hierarchical headings as the\\ncorresponding title. To the best of our knowledge,\\nour work presents the first evaluation of StatPearls\\nin the biomedical NLP community.\\nTextbooks. Textbooks15 (Jin et al., 2021) is\\na collection of 18 widely used medical text-\\nbooks, which are important references for stu-\\ndents taking the United States Medical Li-\\n\\nDocument 2 (Source: 76c4726f8736dba2.pdf):\\nfrom the figure that, in general, the proportion of\\nWikipedia drops in the retrieved snippets for medi-\\ncal questions, which is expected as many snippets\\nin Wikipedia are not related to biomedicine.\\nComparing the distributions for different tasks,\\nthere is a task-specific preference pattern. Med-\\nical examination tasks (MMLU-Med, MedQA-\\nUS, and MedMCQA) tend to have a larger pro-\\nportion of retrieved snippets from Textbooks and\\nStatPearls. PubMedQA* and BioASQ-Y/N with\\nresearch-related questions have more relevant snip-\\npets from PubMed. The Textbooks corpus has\\na larger proportion in MedQA-US than in other\\ndatasets, which can be explained the fact that this\\ncorpus is composed of frequently used textbooks\\nfor the US medical licensing examination.\\n6.4 Practical Recommendations\\nIn this section, we discuss the practical indications\\nand recommendations based on our evaluation re-\\nsults of different MEDRAG settings on MIRAGE .\\nCorpus selection. Results in Table 7 indicate that\\n\\nDocument 3 (Source: 76c4726f8736dba2.pdf):\\npotential solutions of the above-mentioned limita-\\ntions to future work.\\nAcknowledgements\\nGuangzhi Xiong and Aidong Zhang are supported\\nby NIH grant 1R01LM014012 and NSF grant\\n2333740. Qiao Jin and Zhiyong Lu are supported\\nby the NIH Intramural Research Program, National\\nLibrary of Medicine.\\nReferences\\nAsma Ben Abacha, Chaitanya Shivade, and Dina\\nDemner-Fushman. 2019. Overview of the mediqa\\n2019 shared task on textual inference, question en-\\ntailment and question answering. In Proceedings of\\nthe 18th BioNLP Workshop and Shared Task, pages\\n370–379.\\nWaleed Ammar, Dirk Groeneveld, Chandra Bhagavat-\\nula, Iz Beltagy, Miles Crawford, Doug Downey, Ja-\\nson Dunkelberger, Ahmed Elgohary, Sergey Feld-\\nman, Vu Ha, et al. 2018. Construction of the liter-\\nature graph in semantic scholar. In Proceedings of\\nthe 2018 Conference of the North American Chap-\\nter of the Association for Computational Linguistics:\\nHuman Language Technologies, Volume 3 (Industry\\nPapers), pages 84–91.\\n\\n'}, {'role': 'assistant', 'content': 'Thought: The paper is about MEDRAG, a retrieval-augmented generation model for biomedical tasks. It uses a combination of PubMed, StatPearls, and Textbooks corpora.\\nAction: Finish\\nAction Input: The technique used in the paper is Retrieval-Augmented Generation (RAG) with a combination of PubMed, StatPearls, and Textbooks corpora.'}], 'iterations': 2}"""
    
    load_and_print_from_string(data_str)

CONVERSATION ANALYSIS REPORT

📋 FINAL ANSWER:
----------------------------------------
The technique used in the paper is Retrieval-Augmented Generation (RAG) with a combination of PubMed, StatPearls, and Textbooks corpora.

🔄 TOTAL ITERATIONS: 2

💬 CONVERSATION FLOW:
----------------------------------------

[1] 🤖 SYSTEM:
------------------------------

        You are an assistant that solves problems using a step-by-step approach.
You can take the following actions:
- Search: search for relevant information in the document database
- Finish: provi...
(System message truncated for readability)

[2] 👤 USER:
------------------------------
what is the techique that used in this paper?

[3] 🤖 ASSISTANT:
------------------------------
Thought: I need to find out the title and authors of the paper to determine the technique used.
Action: Search
Action Input: "paper title and authors"

[4] 🤖 SYSTEM:
------------------------------
Search Result:
Document 1 (Source: 76c4726f8736dba2.pdf):
tai

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, TextLoader, CSVLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import re
from fastapi.middleware.cors import CORSMiddleware
import shutil
from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Request
from fastapi.responses import JSONResponse
import tempfile
from typing import List
import uvicorn
import threading
from pyngrok import ngrok

# Create document directory
document_directory = "./uploaded_documents"
os.makedirs(document_directory, exist_ok=True)

app = FastAPI()
vector_store = None

# Add CORS middleware to allow cross-origin requests from the Flutter web app
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# [Include all your original functions here: load_documents, split_documents, 
# create_vector_store, generate_rag_with_react, retrieve_context, 
# format_context_for_llm, clean_response, generate_llm_response, 
# parse_action, execute_search, ReActAgent class]


@app.post("/upload-file/")
async def upload_file(file: UploadFile = File(...)):
    """Endpoint to upload a single file, matching Flutter frontend call."""
    global vector_store
    
    try:
        # Create a unique file name to avoid conflicts
        file_extension = os.path.splitext(file.filename)[1].lower()
        file_id = f"{os.urandom(8).hex()}{file_extension}"
        file_path = os.path.join(document_directory, file_id)
        
        # Save the file
        with open(file_path, "wb") as buffer:
            # Read the file in chunks to handle large files
            content = await file.read()
            buffer.write(content)
        
        # Process the document
        documents = load_documents([file_path])
        split_docs = split_documents(documents)
        
        # Initialize or update vector store
        if vector_store is None:
            vector_store = create_vector_store(split_docs)
        else:
            # Add new documents to existing vector store
            vector_store.add_documents(split_docs)
        
        return {
            "file_id": file_id,  # Return a file ID as expected by the frontend
            "original_name": file.filename
        }
    
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error processing file: {str(e)}")

@app.post("/query")
async def query_endpoint(request: Request):
    """Endpoint to handle queries, matching Flutter frontend call."""
    global vector_store
    
    try:
        # Parse JSON body
        body = await request.json()
        query = body.get("query")
        
        if not query:
            raise HTTPException(status_code=400, detail="Query parameter is required")
        
        if vector_store is None:
            return JSONResponse(content={
                "result": "Please upload some documents first before asking questions."
            })
        
        result = generate_rag_with_react(query, vector_store)
        #print(result)
        load_and_print_from_string(result)
       # print_thought_action_process(result)
        return JSONResponse(content={"result": result['final_answer']})  # Match the expected JSON structure
    
    except Exception as e:
        print(f"Query error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Query error: {str(e)}")

# Add a basic health check endpoint
@app.get("/")
def read_root():
    return {"message": "AI Research Assistant API is running"}


def run_server():
    uvicorn.run(app, host="0.0.0.0", port=5027)

if __name__ == "__main__":
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(5027)
    print(f"\n * Public URL: {ngrok_tunnel.public_url}")
    print(f" * Local URL: http://localhost:{8000}\n")
    
    # Start FastAPI server in a separate thread
    server_thread = threading.Thread(target=run_server)
    server_thread.daemon = True
    server_thread.start()
    
    try:
        # Keep main thread alive with a more robust approach
        import time
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Shutting down server...")
        ngrok.kill()


 * Public URL: https://e2e1-34-145-27-23.ngrok-free.app
 * Local URL: http://localhost:8000



INFO:     Started server process [35]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5027 (Press CTRL+C to quit)


INFO:     154.236.93.101:0 - "POST /upload-file/ HTTP/1.1" 200 OK
INFO:     154.236.93.101:0 - "OPTIONS /query HTTP/1.1" 200 OK
CONVERSATION ANALYSIS REPORT

📋 FINAL ANSWER:
----------------------------------------
: The paper "Bioreader: a retrieval-enhanced text-to-text transformer for biomedical literature" by Lorenzo Valgimigli introduces the Bioreader model, which combines a retriever and a generator for improved text-to-text generation in the biomedical domain. The retriever uses Dense Passage Retrieval (DPR) to fetch relevant passages, and the generator is a sequence-to-sequence model conditioned on the retrieved passages. The authors show that Bioreader achieves better performance than other state-of-the-art models in tasks like abstract summarization and question answering.

🔄 TOTAL ITERATIONS: 4

💬 CONVERSATION FLOW:
----------------------------------------

[1] 🤖 SYSTEM:
------------------------------
You are a helpful AI assistant that answers questions by searching through